<a href="https://colab.research.google.com/github/Kowsar-Hossain/Parallel_Processing_Lab/blob/main/phonebook_number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile search_phonebook.cu
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <cstring>
#include <algorithm>
#include <cuda.h>

using namespace std;

#define MAX_STR_LEN 50

// CUDA device function to perform substring check
__device__ bool check(char* str1, char* str2, int len) {
    for (int i = 0; str1[i] != '\0'; i++) {
        int j = 0;
        while (str1[i + j] != '\0' &&
               str2[j] != '\0' &&
               str1[i + j] == str2[j]) {
            j++;
        }
        if (j == len) return true;   // FIXED
    }
    return false;
}

// Kernel: search by PHONE NUMBER
__global__ void searchPhonebook(char* d_names,
                                char* d_numbers,
                                int num_contacts,
                                char* search_number,
                                int number_length) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < num_contacts) {
        char* current_name   = d_names   + idx * MAX_STR_LEN;
        char* current_number = d_numbers + idx * MAX_STR_LEN;

        // 🔹 SEARCH USING NUMBER
        if (check(current_number, search_number, number_length)) {
            printf("%s %s\n", current_name, current_number);
        }
    }
}

int main(int argc, char* argv[]) {

    if (argc != 3) {
        cerr << "Usage: " << argv[0]
             << " <search_phone_number> <threads>\n";
        return 1;
    }

    string search_number = argv[1];
    int num_threads = atoi(argv[2]);

    string file_name = "/content/phonebook1.txt";

    // -------- Read file --------
    vector<string> raw_lines;
    ifstream file(file_name);
    if (!file.is_open()) {
        cerr << "Error opening file\n";
        return 1;
    }

    string line;
    while (getline(file, line)) {
        if (!line.empty()) raw_lines.push_back(line);
    }
    file.close();

    int num_contacts = raw_lines.size();

    // -------- Host memory --------
    char* h_names   = (char*)malloc(num_contacts * MAX_STR_LEN);
    char* h_numbers = (char*)malloc(num_contacts * MAX_STR_LEN);

    for (int i = 0; i < num_contacts; i++) {
        int pos = raw_lines[i].find(",");

        string name   = raw_lines[i].substr(1, pos - 2);
        string number = raw_lines[i].substr(
            pos + 2,
            raw_lines[i].size() - pos - 3
        );

        strncpy(h_names + i * MAX_STR_LEN,
                name.c_str(), MAX_STR_LEN - 1);
        strncpy(h_numbers + i * MAX_STR_LEN,
                number.c_str(), MAX_STR_LEN - 1);

        h_names[i * MAX_STR_LEN + MAX_STR_LEN - 1] = '\0';
        h_numbers[i * MAX_STR_LEN + MAX_STR_LEN - 1] = '\0';
    }

    // -------- Device memory --------
    char *d_names, *d_numbers, *d_search;
    int len = search_number.length();

    cudaMalloc(&d_names,   num_contacts * MAX_STR_LEN);
    cudaMalloc(&d_numbers, num_contacts * MAX_STR_LEN);
    cudaMalloc(&d_search,  len + 1);

    cudaMemcpy(d_names, h_names,
               num_contacts * MAX_STR_LEN,
               cudaMemcpyHostToDevice);

    cudaMemcpy(d_numbers, h_numbers,
               num_contacts * MAX_STR_LEN,
               cudaMemcpyHostToDevice);

    cudaMemcpy(d_search, search_number.c_str(),
               len + 1,
               cudaMemcpyHostToDevice);

    // -------- Kernel launch --------
    for (int i = 0; i < num_contacts; i += num_threads) {
        int threads = min(num_contacts - i, num_threads);

        searchPhonebook<<<1, threads>>>(
            d_names   + i * MAX_STR_LEN,
            d_numbers + i * MAX_STR_LEN,
            threads,
            d_search,
            len
        );
        cudaDeviceSynchronize();
    }

    // -------- Cleanup --------
    free(h_names);
    free(h_numbers);
    cudaFree(d_names);
    cudaFree(d_numbers);
    cudaFree(d_search);

    return 0;
}


Writing search_phonebook.cu


In [3]:
!nvcc  -arch=sm_75 search_phonebook.cu -o search_phonebook

In [5]:
!time ./search_phonebook 665 10 > output.txt


real	0m0.229s
user	0m0.018s
sys	0m0.206s
